# Natural Language Command & Control for UAV's

## Assignment 2 Solutions

**Marut Garg (240633)**  
**IIT Kanpur – Electrical Engineering**  
**December 24, 2025**


## Q1. Computational Graphs & Gradients

**Gradient Accumulation in PyTorch**

In PyTorch, gradients are accumulated by default. When we call `loss.backward()`, the computed gradients are **added** to the existing values stored in the `.grad` attribute of leaf tensors. The old gradients are not overwritten automatically.

In a standard training loop, `optimizer.zero_grad()` is used to reset all gradients to zero before calculating new gradients. If we forget to do this, gradients from previous iterations will keep accumulating.

Mathematically, this means that parameter updates will be based on the sum of gradients from multiple steps, which leads to incorrect updates and unstable training.



## Q2. Tensors: View vs Reshape

Both `.view()` and `.reshape()` are used to change the shape of a tensor in PyTorch.

The main technical difference is related to memory layout. The `.view()` method works only when the tensor is stored in a contiguous block of memory. If the tensor is non-contiguous, calling `.view()` will result in an error.

On the other hand, `.reshape()` is more flexible. If the tensor is not contiguous, `.reshape()` automatically creates a new contiguous tensor if needed.

Therefore, `.reshape()` is safer to use when we are unsure about the memory layout or stride of the tensor.


## Q3. Device Management (CPU vs GPU)

If we try to perform an operation such as addition between a tensor on the CPU and a tensor on the GPU, PyTorch gives an error. This happens because PyTorch does not allow operations between tensors that are on different devices. Both tensors must be on the same device.

When a model is moved to the GPU using `model.to('cuda')`, all the model parameters are moved to the GPU. However, any new tensors created inside the `forward` method do not automatically move to the GPU. These tensors must be explicitly created on the same device, otherwise a device mismatch error will occur.



## Q4. Tensor Manipulation (The Image Mask)

In [ ]:
import torch

def process_images(images):
    masked_images = torch.where(images < 0.5, 0.0, 1.0)
    return masked_images

images = torch.rand(4, 28, 28)
output = process_images(images)

output


## Q5. The "Safe" Autograd (Gradient Checking)

In [ ]:
import torch

# Initialize x with gradient tracking
x = torch.tensor(4.0, requires_grad=True)

# Define the function y = x^3 + 2x
y = x**3 + 2*x

# Backward pass to compute gradient
y.backward()

# Print gradient computed by PyTorch
print("Gradient from PyTorch:", x.grad)

# Manual gradient calculation: dy/dx = 3x^2 + 2
manual_grad = 3*(4**2) + 2

# Sanity check
if x.grad.item() != manual_grad:
    raise ValueError("Gradient does not match manual calculation")

print("Manual gradient:", manual_grad)



## Q6. Class Interaction (Custom Datasets)

In [ ]:
import torch
from torch.utils.data import Dataset

class NumberDataset(Dataset):
    def __init__(self, numbers):
        self.numbers = numbers

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, idx):
        input_tensor = torch.tensor(float(self.numbers[idx]))
        target_tensor = torch.tensor(float(self.numbers[idx] * 2))
        return input_tensor, target_tensor

dataset = NumberDataset([1, 2, 3, 4])

print(len(dataset))
print(dataset[1])
print(dataset[3])


## Q7. Model Encapsulation (nn.Module)

In [ ]:
import torch
import torch.nn as nn

class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = SimpleClassifier()
dummy_input = torch.rand(2, 10)
output = model(dummy_input)

print(output)


## Q8. The Training Loop (Logic Integration)

In [ ]:
def train_step(model, inputs, targets, optimizer, criterion):
    # Forward pass
    predictions = model(inputs)
    
    # Loss calculation
    loss = criterion(predictions, targets)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Return scalar loss value
    return loss.item()

import torch
import torch.nn as nn

# Create model
model = SimpleClassifier()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Loss function
criterion = nn.BCELoss()

# Dummy inputs and targets
inputs = torch.rand(4, 10)
targets = torch.rand(4, 1)

# Run one training step
loss = train_step(model, inputs, targets, optimizer, criterion)

print("Loss:", loss)
